# Setup Data and Analysis

In [12]:
# Import config and libraries
import config as c
import chart_studio
import chart_studio.plotly as py
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import chart_studio.tools as tls
import pandas as pd
import numpy as np
import urllib, json
import datetime as dt
from datetime import timedelta
import matplotlib
import matplotlib.pyplot as plt
import plotly.express as px

#API key to Chart studio
chart_studio.tools.set_credentials_file(username=c.cs_un, api_key=c.cs_key)

In [13]:
#assign global variables
week_ending_date = '2020-07-11'

In [14]:
#Florida Department of Health: Coronavirus source data
url = "https://opendata.arcgis.com/datasets/37abda537d17458bae6677b8ab75fcb9_0.geojson"

In [15]:
#Create pandas dataframe from Florida DOH dataset
json_url = urllib.request.urlopen(url)
data = json.loads(json_url.read())
L = []
for x in data['features']:
    d = {}
    for k, v in x.items():
        if k == 'properties':
            for x, y in v.items():
                d[x] = y
        else:
            pass
    L.append(d)

df = pd.DataFrame(L)

In [16]:
#Convert Died boolean to a binary integer
df['Died_int']=np.where( df['Died'] == "NA",0, 1)

#Fill missing ages with 999. Only a handful of such cases exist
df['Age']=np.where( df['Age'] == 'NA',999, df['Age'])
df = df[df.Age != 999]

#Fix data formats
df = df.astype({'Age': 'int32'})
df['ChartDate'] = df['ChartDate'].apply( lambda x: pd.to_datetime(x)).dt.normalize()
df['DateWeek'] = df.apply(lambda row: row['ChartDate'] - dt.timedelta(days=row['ChartDate'].weekday()+1), axis=1)
df['DateMonth'] = df['ChartDate'].apply(lambda x: x.strftime('%M-%Y'))

In [19]:
#Create bins based age groups
df['Age']=np.maximum(np.minimum(df['Age'],100),1)
bins2 = [0, 25, 50, 60, 70, 80, 100]
df['binned'] = pd.cut(df['Age'], bins=bins2)

bins3 = [0, 50, 70, 100]
df['binned_condensed'] = pd.cut(df['Age'], bins=bins3)

### 1. Case volumes and death rates

In [13]:
#1A Case volumes by age group and reported date

age = df[df['ChartDate']<=week_ending_date].pivot_table(values=['Case1'], 
                      index=['DateWeek'],
                      columns=['binned_condensed'],
                      aggfunc='count')

# age = age.div(age.sum(1), axis=0)
age#.plot(kind='bar', stacked=True)

Case1                   
binned_condensed (-1, 50] (50, 70] (70, 120]
DateWeek                                    
2020-03-01              2        7         5
2020-03-08             52       52        19
2020-03-15            467      331       170
2020-03-22           1982     1247       662
2020-03-29           3771     2561      1157
2020-04-05           3626     2551      1246
2020-04-12           3135     2001      1113
2020-04-19           2891     1688      1104
2020-04-26           2273     1454      1040
2020-05-03           2080     1137       886
2020-05-10           2917     1509       962
2020-05-17           3226     1332       800
2020-05-24           3278     1298       631
2020-05-31           5216     1829       809
2020-06-07           8750     2486      1021
2020-06-14          16963     4278      1508
2020-06-21          35024     8860      2884
2020-06-28          43035    12655      3816
2020-07-05          43748    14937      5012

In [15]:
#1A Case volumes by age group and reported date - Plot

df2 = df.groupby(['DateWeek','binned']).agg({'Case1':'count'}).reset_index()

fig = px.bar(df2,
             x="DateWeek",
             y="Case1",
             color='binned',
             barmode='stack',
             labels = {'DateWeek':'Date','Case1':'Confirmed Case Volume','binned':'Age Group'},
             title={
                'text': "Weekly COVID-19 Case Volumes by Age Group - Florida",
                'y':0.9,
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'})

fig.update_yaxes(tickformat = ',.0')
fig.update_traces(hovertemplate=None)
fig.update_layout(hovermode="x")
config = {'responsive': False}

fig.show()
py.plot(fig, filename = 'Weekly COVID-19 Case Volumes by Age Group - Florida', auto_open=False, conifg=config)

'https://plotly.com/~natefrey93/1/'

In [2]:
tls.get_embed('https://plotly.com/~natefrey93/1/')

'<iframe id="igraph" scrolling="no" style="border:none;" seamless="seamless" src="https://plotly.com/~natefrey93/1.embed" height="525" width="100%"></iframe>'

In [11]:
#1B Death volumes by age group and case reported date

age = df.pivot_table(values=['Died_int'], 
                      index=['DateWeek'],
                      #columns=['binned'],
                      aggfunc='sum')

# age = age.div(age.sum(1), axis=0)
age#.plot(kind='bar', stacked=True)

,Died_int
DateWeek,
2020-03-01,2
2020-03-08,12
2020-03-15,62
2020-03-22,254
2020-03-29,384
2020-04-05,432
2020-04-12,418
2020-04-19,365
2020-04-26,317


In [12]:
#1D Death rates by age group over time (graph)
df['DateMonth'] = df['ChartDate'].dt.to_period('M')
df['Died_perc']=df['Died_int']*100

death_rate = df[df['ChartDate']<='2020-06-20'].pivot_table(values=['Died_perc'], 
                      index=['DateMonth'],
                      columns=['binned'],
                      aggfunc='mean')

# age = age.div(age.sum(1), axis=0)
death_rate#.plot(kind='line', legend=False)

Died_perc                                                    
binned     (-1, 25]  (25, 50]  (50, 60]  (60, 70]   (70, 80]  (80, 120]
DateMonth                                                              
2020-03    0.000000  1.015965  2.974277  8.191808  18.005181  38.611111
2020-04    0.000000  0.758978  2.158843  6.944096  17.174836  33.376178
2020-05    0.056497  0.310421  1.881246  5.960532  15.375854  28.657921
2020-06    0.009120  0.189955  1.018981  2.742409   7.026713  18.871866

In [13]:
#1A Case volumes by age group and reported date
import plotly.graph_objects as go

df2 = df[(df['DateWeek']<='2020-06-14')&(df['DateWeek']>='2020-03-15')].groupby(['DateWeek','binned']).agg({'Died_int':'mean'}).reset_index()

fig = px.line(df2,
             x="DateWeek",
             y="Died_int",
             color='binned',
             #barmode='stack',
             labels = {'DateWeek':'Date','Died_int':'Death Rate','binned':'Age Group'},
             title={
                'text': "Average Weekly Death Rate by Case Reported Date - Florida",
                'y':0.9,
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'})

fig.update_yaxes(tickformat = ',.1%')
fig.update_traces(hovertemplate=None)
fig.update_layout(hovermode="x")

fig.show()
#py.plot(fig, filename = 'Average Weekly Death Rate by Case Reported Date - Florida', auto_open=False)

### 2.Death Prediction

In [14]:
#2A Predicted deaths by age group and date
df2=df

#Theoretical death reported date is equal to case date + 14 days for average death lag and +7 days for average death reporting lag
df2['tdr_date'] = df2['ChartDate'] + timedelta(days=14) + timedelta(days=7)
df2['TDR_Week'] = df2.apply(lambda row: row['tdr_date'] - dt.timedelta(days=row['tdr_date'].weekday()+1), axis=1)

#Create a dictionary with death rate assumptions based on first 3 weeks of June, merge to df
df2['avg_death_rate']=df2['Died_int']
dr_df = pd.pivot_table(df2[(df2['ChartDate']>='2020-06-01') & (df2['ChartDate']<='2020-06-21') ],index=['binned'],values=["avg_death_rate"],aggfunc=np.mean, margins=True)
dr_df.drop(index='All', axis=0).reset_index()
df3 = df2.merge(dr_df, on='binned', how='left',suffixes=('_l', '_r'))

In [16]:
df4 = df3.groupby(['TDR_Week','binned']).agg({'avg_death_rate_r':'sum'}).reset_index()

In [17]:
df4 = df4.astype({'avg_death_rate_r':int}) 

fig = px.bar(df4,
             x="TDR_Week",
             y='avg_death_rate_r',
             color='binned',
             barmode='stack',
             labels = {'TDR_Week':'Date','avg_death_rate_r':'Predicted Death Count','binned':'Age Group'},
             title={
                'text': "Weekly Actual and Forecasted COVID-19 Death Volumes by Age Group - Florida",
                'y':0.9,
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'})

temp = {'TDR_Week': ['2020-07-08','2020-07-08'],
        'avg_death_rate_r': [-10000, 10000],
        'binned':['7/12/2020','7/12/2020']
        }
temp_df = pd.DataFrame(temp, columns = ['TDR_Week', 'avg_death_rate_r','binned'])


fig2 = px.line(temp_df
                ,x='TDR_Week'
                ,y='avg_death_rate_r'
               ,color='binned'
            ,color_discrete_sequence=["black"]
               ,hover_data={'avg_death_rate_r':False
                           ,'TDR_Week':False}
               ,labels = {'TDR_Week':'<--- Actuals | Predictions--->','avg_death_rate_r':'blah','binned':'Forecasted After Date'},
)

fig.add_trace(fig2.data[0])
fig.update_yaxes(tickformat = ',.0', range = [0,900])
fig.update_traces(hovertemplate=None)
fig.update_layout(hovermode="x")

#fig.add_bar(df2,x='TDR_Week', y='pred_deaths', barmode='stacked')


fig.show()
#py.plot(fig, filename = 'Weekly Actual and Forecasted COVID-19 Death Volumes by Age Group - Florida', auto_open=False)

### 3. Scenario Analysis

In [53]:
df_agg=df3.groupby(['DateWeek','binned']).agg({'Case1':'count','avg_death_rate_r': 'mean'}).reset_index()
df_agg['weekly_growth_rate']=df_agg['Case1'].div(df_agg['Case1'].shift(6))

In [54]:
May_temp = df_agg[(df_agg['DateWeek']>='2020-05-03') & (df_agg['DateWeek']<'2020-05-31')]
May_temp=May_temp.groupby('binned').agg({'Case1':'mean','avg_death_rate_r': 'mean'}).reset_index()
May_temp['DateWeek']='2020-07-05'

May_temp2=May_temp.rename(columns={'Case1':'May_avg_cases'})
May_temp2=May_temp2[['DateWeek','binned','May_avg_cases']]
May_temp2['DateWeek'] = May_temp2['DateWeek'].apply( lambda x: pd.to_datetime(x)).dt.normalize()

temp = df_agg[df_agg['DateWeek']=='2020-07-05'].merge(May_temp2, on=['DateWeek','binned'], how='left')
temp['SceA_cases']=temp['Case1']

In [55]:
for x in range(7):
    temp.loc[:,'DateWeek'] = temp['DateWeek'] + dt.timedelta(days=7)
    temp['weekly_growth_rate']=temp['weekly_growth_rate']*.95 #Assume weekly growth rate tapers off by 7% per week
    temp.loc[:,'SceA_cases'] = temp['SceA_cases']*temp['weekly_growth_rate']
    temp['SceA_deaths'] = 0
    temp.loc[:,'SceB_cases'] = ((temp['Case1']+temp['May_avg_cases'])/2*x+temp['Case1']*(3-x))/3 #Drop case volume to midpoint of May lows and current highs by August 1st
    temp['SceB_deaths'] = 0
    temp['SceC_cases'] = (temp['May_avg_cases']*x+temp['Case1']*(3-x))/3 #Return to May low case loads by August 1st
    temp['SceC_deaths'] = 0
    df_agg = df_agg.append(temp)

df_agg=df_agg.fillna({'SceA_cases': 0.0, 'SceB_cases': 0.0, 'SceC_cases': 0.0, 'SceA_deaths': 0.0,'SceB_deaths': 0.0,'SceC_deaths': 0.0})

df_agg.loc[:,'SceA_deaths'] = np.where(df_agg['DateWeek']<='2020-08-01',df_agg['avg_death_rate_r'].mul(df_agg['Case1'].shift(18)),df_agg['avg_death_rate_r'].mul(df_agg['SceA_cases'].shift(18)))
df_agg.loc[:,'SceB_deaths'] = np.where(df_agg['DateWeek']<='2020-08-01',df_agg['avg_death_rate_r'].mul(df_agg['Case1'].shift(18)),df_agg['avg_death_rate_r'].mul(df_agg['SceB_cases'].shift(18)))
df_agg.loc[:,'SceC_deaths'] = np.where(df_agg['DateWeek']<='2020-08-01',df_agg['avg_death_rate_r'].mul(df_agg['Case1'].shift(18)),df_agg['avg_death_rate_r'].mul(df_agg['SceC_cases'].shift(18)))

In [56]:
#3A Death predictions by Scenario
pred_deaths = df_agg.pivot_table(values=['SceA_deaths','SceB_deaths','SceC_deaths'], 
                      index=['DateWeek'],
                      #columns=['SceA_deaths'],
                      aggfunc='sum')

# age = age.div(age.sum(1), axis=0)
pred_deaths#.plot(kind='line')

,SceA_deaths,SceB_deaths,SceC_deaths
DateWeek,,,
2020-03-01,0.0,0.0,0.0
2020-03-08,0.0,0.0,0.0
2020-03-15,0.0,0.0,0.0
2020-03-22,1.4,1.4,1.4
2020-03-29,3.3,3.3,3.3
2020-04-05,26.0,26.0,26.0
2020-04-12,97.8,97.8,97.8
2020-04-19,181.1,181.1,181.1
2020-04-26,205.7,205.7,205.7


In [57]:
df5 = df_agg.groupby(['DateWeek']).agg({'SceA_deaths':'sum','SceB_deaths':'sum','SceC_deaths':'sum'}).reset_index()
df5 = df5.astype({'SceA_deaths':int,'SceB_deaths':int,'SceC_deaths':int}) 

df5['Scenario']="A"
figA = px.line(df5,
             x="DateWeek",
             y='SceA_deaths',
             labels = {'DateWeek':'Date','SceA_deaths':'Death Volume'},
            color='Scenario',
            color_discrete_sequence=["red"],
             title={
                'text': "Scenario Analysis - Actual and Forecasted Weekly Death Volumes",
                'y':0.9,
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'})
df5['Scenario']="B"
figB = px.line(df5,
             x="DateWeek",
             y='SceB_deaths',
             labels = {'DateWeek':'Date','SceB_deaths':'Death Volume'},
               color='Scenario',
               color_discrete_sequence=["green"],
             title={
                'text': "Scenario Analysis - Actual and Forecasted Weekly Death Volumes",
                'y':0.9,
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'})
df5['Scenario']="C"
figC = px.line(df5,
             x="DateWeek",
             y='SceC_deaths',
             labels = {'DateWeek':'Date','SceC_deaths':'Death Volume'},
               color='Scenario',
               color_discrete_sequence=["blue"],
             title={
                'text': "Scenario Analysis - Actual and Forecasted Weekly Death Volumes",
                'y':0.9,
                'x':0.5,
                'xanchor': 'center',
                'yanchor': 'top'})

temp = {'DateWeek': ['2020-07-08','2020-07-08'],
        'SceA_deaths': [-10000.00, 10000.00],
        'binned':['7/12/2020','7/12/2020']
                }
temp_df = pd.DataFrame(temp, columns = ['DateWeek', 'SceA_deaths','binned'])


fig_vertical = px.line(temp_df
                ,x='DateWeek'
                ,y='SceA_deaths'
                ,color='binned'
                ,color_discrete_sequence=["black"]
               ,labels = {'DateWeek':'Date','SceA_deaths':'blah','binned':'Forecasted After Date'},
)

figA.add_trace(figB.data[0])
figA.add_trace(figC.data[0])
figA.add_trace(fig_vertical.data[0])
figA.update_traces(hovertemplate=None)
figA.update_layout(hovermode="x")
figA.update_yaxes(tickformat = ',.0', range = [0,3000])

#fig.add_bar(df2,x='TDR_Week', y='pred_deaths', barmode='stacked')


figA.show()
#py.plot(figA, filename = 'Scenario Analysis - Actual and Forecasted Weekly Death Volumes', auto_open=False)

'https://plotly.com/~natefrey93/9/'

In [1]:
tls.get_embed('https://plotly.com/~natefrey93/9/')

'<iframe id="igraph" scrolling="no" style="border:none;" seamless="seamless" src="https://plotly.com/~natefrey93/9.embed" height="525" width="100%"></iframe>'

In [58]:
#3B Economic costs
df_agg['DateMonth'] = df_agg['DateWeek'].apply(lambda x: x.strftime('%M-%Y')) 
df_agg2=df_agg[(df_agg['DateWeek']>='2020-08-01') & (df_agg['DateWeek']<'2020-08-25')].groupby('DateMonth').agg({'SceA_deaths':'sum','SceB_deaths': 'sum','SceC_deaths': 'sum'})
pt = df_agg2.pivot_table(values=['SceA_deaths','SceB_deaths','SceC_deaths'], 
                      index=['DateMonth'],
                      aggfunc='sum')

pd.options.display.float_format = '{:.1f}'.format

pt2=pt.reset_index().rename(columns={'DateMonth':'Scenario','SceA_deaths':'A: Restart the normal economy','SceB_deaths':'B: Transition to a low-touch economy','SceC_deaths':'C: Re-instate April/May lockdown'}).T.reset_index().drop(index=0)
pt2=pt2.rename(columns={'index':'Scenario',0:'Forecasted Death Count'})
pt2['Statistical Human Cost ($B)']=pt2['Forecasted Death Count']/100
pt2['Florida GDP ($B)']=950.76
pt2['GDP Cost ($B)'] = np.where(pt2['Scenario'].astype(str).str[0]=='C',pt2['Florida GDP ($B)']*.05,
                                np.where(pt2['Scenario'].astype(str).str[0]=='B',pt2['Florida GDP ($B)']*.025,0))
pt2['Difference']=pt2['Florida GDP ($B)']-pt2['Statistical Human Cost ($B)']
pt2['Breakeven Reduction in Economic Activity ($B)'] = pt2['Difference'] - pt2.iloc[0,4]
pt2['Total Economic Cost ($B)']=pt2['GDP Cost ($B)']+pt2['Statistical Human Cost ($B)']
pt2=pt2[['Scenario','Forecasted Death Count','Statistical Human Cost ($B)','Florida GDP ($B)','GDP Cost ($B)','Total Economic Cost ($B)']]
pt2.set_index('Scenario')

,Forecasted Death Count,Statistical Human Cost ($B),Florida GDP ($B),GDP Cost ($B),Total Economic Cost ($B)
Scenario,,,,,
A: Restart the normal economy,11128.3,111.3,950.8,0.0,111.3
B: Transition to a low-touch economy,3378.8,33.8,950.8,23.8,57.6
C: Re-instate April/May lockdown,2435.8,24.4,950.8,47.5,71.9


In [60]:
import plotly.graph_objects as go

df6 = pt2

fig = go.Figure(data=[go.Table(
    header=dict(values=list(df6.columns),
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[df6['Scenario'],df6['Forecasted Death Count'],df6['Statistical Human Cost ($B)'],df6['Florida GDP ($B)'],df6['GDP Cost ($B)'],df6['Total Economic Cost ($B)']],
               fill_color='lavender',
               align='left',
               format=[None,',.0f','$.2f']))
])

fig.update_layout(title_text="Total Economic Cost - August 2020 Forecasts, Florida")

fig.show()
#py.plot(fig, filename = 'Breakeven Analysis by Scenario - August 2020 Forecasts', auto_open=False)

'https://plotly.com/~natefrey93/22/'